In [52]:
import numpy as np; import matplotlib.pyplot as plt

direc = "/home/callum/code/spectra/out/NLLZ/7_PROD_2/2/1000"
cvac = 299792458.0
ns = 2048

wn = np.array([i / (ns * cvac * 100 * 1E-15) for i in range(ns)])

def plt_overlap(direc, car_index, chl_index):
    car_file = "{}/car_{:02d}.dat".format(direc, car_index)
    chl_file = "{}/chl_{:02d}.dat".format(direc, chl_index)
    car = np.loadtxt(car_file)
    chl = np.loadtxt(chl_file)
    #print(np.trapz(car, x=wn), 1./np.trapz(car, x=wn))
    #print(np.trapz(chl[:, 0], x=wn), np.trapz(chl[:, 1], x=wn))
    car_to_chl = car * chl[:, 0]
    chl_to_car = car * chl[:, 1]
    fig, ax = plt.subplots(2, sharex=True)
    ax[0].plot(wn, car, label=r'car')
    ax[0].plot(wn, chl[:, 0], label=r'chl $A(\omega)$')
    ax[0].plot(wn, chl[:, 1], label=r'chl $F(\omega)$')
    car_to_chl_rate = np.trapz(car_to_chl)
    chl_to_car_rate = np.trapz(chl_to_car)
    ax[1].plot(wn, car_to_chl, label=r'car $ \rightarrow $ chl, overlap = {:10.6f}'.format(car_to_chl_rate))
    ax[1].plot(wn, chl_to_car, label=r'chl $ \rightarrow $ car, overlap = {:10.6f}'.format(chl_to_car_rate))
    ax[0].legend(); ax[1].legend()
    ax[1].set_xlabel("Wavenumber $ (cm^{-1}) $")
    ax[0].set_xlim([10000,20000])
    plt.savefig("{}/car_{:02d}_chl_{:02d}.pdf".format(direc, car_index, chl_index))
    #plt.show()
    plt.close(fig)

    

# note that these rates won't match the C code - i'm not calculating the couplings or FC overlaps here
#for car_index in range(16,32):
#    for chl_index in range(14):
#       plt_overlap(direc, car_index, chl_index)


plt_overlap(direc, 0, 0)